# EDA — Poêles à pellets & météo
Objectif : construire les hypothèses et les vérifications nécessaires pour relier météo, cycles d'allumage et consommation de granulés.


## Plan express
1. Charger et inspecter la structure des données (météo + cycles utilisateur)
2. Formuler des hypothèses (ex. température ↔ consommation)
3. Préparer les features temporelles et météo
4. Visualiser les distributions clés
5. Tester les relations (corrélations, comparaisons de moyennes)
6. Synthétiser les enseignements opérationnels


In [ ]:
# Installer les dépendances manquantes (exécuté dans le notebook)
%pip install scipy

# Imports principaux
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import stats

sns.set_theme(style="whitegrid")
pd.set_option("display.max_columns", 50)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Paramètres à adapter en fonction de la localisation du fichier source
RAW_DATA_PATH = Path("data/raw/pellet_cycles.csv")

# Colonnes attendues (ajuster selon le jeu de données réel)
TIME_COL = "timestamp"
CONS_COL = "pellet_kg"
TEMP_COL = "temperature_ext"
HUM_COL = "humidite"
WIND_COL = "vent_kmh"
PRICE_COL = "prix_euro"
TARGET_TEMP_COL = "temperature_int_cible"


In [ ]:
# Chargement des données avec fallbacks et échantillon synthétique si besoin
from datetime import timedelta


def _workspace_roots() -> list[Path]:
    roots: list[Path] = []
    cwd = Path.cwd().resolve()
    for candidate in (cwd, cwd.parent, cwd.parent.parent):
        if candidate not in roots:
            roots.append(candidate)
    file_hint = globals().get("__file__")
    if file_hint:
        repo_hint = Path(file_hint).resolve().parent.parent
        if repo_hint not in roots:
            roots.append(repo_hint)
    return roots


def _expand_candidates(raw: Path) -> list[Path]:
    raw = raw.expanduser()
    if raw.is_absolute():
        return [raw]
    options: list[Path] = []
    for base in _workspace_roots():
        options.append((base / raw).resolve())
    return options


def load_cycles(primary: Path, fallbacks: list[Path]) -> pd.DataFrame:
    for path in [primary, *fallbacks]:
        if path is None:
            continue
        for candidate in _expand_candidates(path):
            if not candidate.exists():
                continue
            try:
                df_loaded = pd.read_csv(candidate, parse_dates=[DATE_COL])
            except Exception as exc:  # pragma: no cover
                print(f"Impossible de lire {candidate}: {exc}")
                continue
            print(f"Jeu de données chargé depuis {candidate}")
            return df_loaded
    print("Aucune source réelle trouvée, génération d'un échantillon synthétique (5 jours).")
    base = pd.Timestamp("2025-01-01")
    sample = {
        DATE_COL: [base + timedelta(days=i) for i in range(5)],
        TEMP_AVG_COL: [2.5, 1.0, -1.2, 0.5, 3.0],
        TEMP_MIN_COL: [-2.0, -3.5, -5.0, -4.0, -1.5],
        TEMP_MAX_COL: [6.0, 4.0, 2.0, 5.0, 7.0],
        WIND_COL: [12, 15, 8, 5, 10],
        DURATION_COL: [4.0, 5.5, 6.0, 3.0, 2.5],
        CONS_COL: [0.35, 0.42, 0.5, 0.3, 0.25],
        PRICE_COL: [1.7, 2.0, 2.4, 1.5, 1.2],
    }
    return pd.DataFrame(sample)


df = load_cycles(RAW_DATA_PATH, FALLBACK_PATHS)

# Post-traitements génériques
if DATE_COL not in df.columns:
    raise ValueError(f"La colonne {DATE_COL} est obligatoire pour l'analyse.")

df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
df = df.dropna(subset=[DATE_COL]).sort_values(DATE_COL).reset_index(drop=True)

df["pellet_kg"] = df[CONS_COL] * KG_PER_BAG

# Colonnes disponibles pour les prochains graphiques
available_cols = set(df.columns)
print(f"Colonnes détectées : {sorted(available_cols)}")
df.head()


NameError: name 'FALLBACK_PATHS' is not defined

## 1) Structure & qualité des données
Vérifier la complétude, les types, et la granularité temporelle.


In [ ]:
# Aperçu structurel
info_df = df.info()
display(info_df)

# Statistiques descriptives brutes
display(df.describe(datetime_is_numeric=True).T)

# Taux de valeurs manquantes
missing_rate = df.isna().mean().sort_values(ascending=False)
display(missing_rate.to_frame("missing_ratio"))


## 2) Hypothèses clés à vérifier
- H1 : plus la température extérieure est basse, plus la consommation de pellets augmente.
- H2 : le vent et l'humidité amplifient les pics de consommation (pertes thermiques + confort ressenti).
- H3 : les cycles du matin sont plus longs/coûteux que ceux du soir à température extérieure équivalente.
- H4 : la surconsommation apparaît quand la consigne intérieure est trop élevée par rapport à l'inertie du logement.


In [ ]:
# Préparation des features temporelles
series_date = pd.to_datetime(df[DATE_COL], errors="coerce")
df["jour_semaine"] = series_date.dt.day_name()
df["mois"] = series_date.dt.month
df["annee"] = series_date.dt.year
df["saison"] = pd.cut(
    df["mois"],
    bins=[0, 3, 6, 9, 12],
    labels=["Hiver", "Printemps", "Été", "Automne"],
    include_lowest=True,
)

# Agrégations rapides (déjà quotidiennes mais pratique pour rajouter des features)
daily = df[[DATE_COL, CONS_COL, "pellet_kg", TEMP_AVG_COL, TEMP_MIN_COL, TEMP_MAX_COL, WIND_COL, DURATION_COL, PRICE_COL]].copy()
daily = daily.rename(
    columns={
        CONS_COL: "pellet_bags",  # harmonisation des intitulés
        TEMP_AVG_COL: "temp_avg_c",
        TEMP_MIN_COL: "temp_min_c",
        TEMP_MAX_COL: "temp_max_c",
        WIND_COL: "wind_avg_kmh",
        DURATION_COL: "duration_hours",
        PRICE_COL: "pellet_cost_eur",
    }
)

daily["jour_semaine"] = df["jour_semaine"]
daily["saison"] = df["saison"]

daily.head()


## 3) Distributions de base
Objectif : détecter asymétries, valeurs aberrantes et niveaux usuels.


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

sns.histplot(df["pellet_kg"], kde=True, ax=axes[0, 0], color="#6A4C93")
axes[0, 0].set_title("Distribution consommation (kg)")
axes[0, 0].set_xlabel("Pellets (kg)")

sns.histplot(df[TEMP_AVG_COL], kde=True, ax=axes[0, 1], color="#1982C4")
axes[0, 1].set_title("Distribution température moyenne")
axes[0, 1].set_xlabel("Température moyenne (°C)")

if df["saison"].notna().any():
    sns.boxplot(x="saison", y="pellet_kg", data=df, ax=axes[1, 0], palette="muted")
    axes[1, 0].set_title("Conso par saison")
else:
    axes[1, 0].set_visible(False)

sns.boxplot(x="jour_semaine", y="pellet_kg", data=df, ax=axes[1, 1], palette="coolwarm")
axes[1, 1].set_title("Conso par jour de la semaine")
plt.setp(axes[1, 1].get_xticklabels(), rotation=45)

plt.tight_layout()
plt.show()


## 4) Relation météo ↔ consommation (H1 + H2)
- Visualisation : nuage de points + régression locale
- Mesure : corrélation de Spearman (robuste aux non-linéarités)


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.regplot(x=TEMP_MIN_COL, y="pellet_kg", data=df, lowess=True, ax=axes[0], scatter_kws={"alpha": 0.4})
axes[0].set_title("Température min vs consommation")
axes[0].set_xlabel("Température min (°C)")
axes[0].set_ylabel("Pellets (kg)")

sns.regplot(x=TEMP_MAX_COL, y="pellet_kg", data=df, lowess=True, ax=axes[1], scatter_kws={"alpha": 0.4}, color="#FF595E")
axes[1].set_title("Température max vs consommation")
axes[1].set_xlabel("Température max (°C)")
axes[1].set_ylabel("")

sns.regplot(x=WIND_COL, y="pellet_kg", data=df, lowess=True, ax=axes[2], scatter_kws={"alpha": 0.4}, color="#8AC926")
axes[2].set_title("Vent vs consommation")
axes[2].set_xlabel("Vent moyen (km/h)")
axes[2].set_ylabel("")

plt.tight_layout()
plt.show()

meteo_cols = [col for col in [TEMP_AVG_COL, TEMP_MIN_COL, TEMP_MAX_COL, WIND_COL] if col in df.columns]
corr_spearman = df[["pellet_kg"] + meteo_cols].corr(method="spearman")
corr_spearman["pellet_kg"]


## 5) Effet horaire & cycles (H3)
Comparer les profils matin (6h-10h) vs soir (18h-22h) à température extérieure comparable.


In [ ]:
df["type_cycle"] = pd.cut(
    df[DURATION_COL],
    bins=[0, 2, 4, 6, 24],
    labels=["Très court", "Court", "Standard", "Long"],
    include_lowest=True,
)

fig, ax = plt.subplots(figsize=(10, 5))
sns.boxplot(x="type_cycle", y="pellet_kg", data=df, ax=ax, palette="pastel")
ax.set_title("Consommation par durée de cycle")
ax.set_xlabel("Durée d'un cycle (h)")
ax.set_ylabel("Pellets (kg)")
plt.show()

short_cycles = df[df["type_cycle"] == "Court"]["pellet_kg"].dropna()
long_cycles = df[df["type_cycle"] == "Long"]["pellet_kg"].dropna()
if len(short_cycles) > 2 and len(long_cycles) > 2:
    t_stat, p_val = stats.ttest_ind(short_cycles, long_cycles, equal_var=False)
    print(f"Test t Court vs Long — statistique={t_stat:.2f}, p-value={p_val:.3g}")
else:
    print("Pas assez de cycles dans les catégories Court/Long pour un test statistique fiable.")


## 6) Surchauffe potentielle (H4)
Approche : comparer la consommation aux écarts entre température intérieure cible et température extérieure.


In [ ]:
# Approche proxy : plus l'écart à la température de confort (21°C) est élevé, plus on s'attend à consommer
CONFORT_CIBLE = 21

df["delta_confort"] = CONFORT_CIBLE - df[TEMP_AVG_COL]
df["kg_par_heure"] = df["pellet_kg"] / df[DURATION_COL]

fig, ax = plt.subplots(figsize=(8, 5))
sns.regplot(x="delta_confort", y="kg_par_heure", data=df, lowess=True, ax=ax, scatter_kws={"alpha": 0.4})
ax.set_title("Écart à 21°C vs intensité des cycles")
ax.set_xlabel("Delta confort (°C)")
ax.set_ylabel("Pellets par heure (kg/h)")
plt.show()

corr_delta = df[["delta_confort", "kg_par_heure"]].corr(method="spearman").iloc[0, 1]
print(f"Corrélation Spearman delta confort ↔ kg/h : {corr_delta:.3f}")


## 7) Coût & efficacité
Conversion de la consommation en € et recherche de surconsommation évitable.


In [ ]:
if PRICE_COL in df.columns:
    df["cout_cycle"] = df[PRICE_COL]
    df["prix_par_kg"] = df["cout_cycle"] / df["pellet_kg"].replace(0, np.nan)
    daily_costs = df.groupby(df[DATE_COL].dt.to_period("M"))["cout_cycle"].sum()
    print(f"Coût quotidien moyen : {df['cout_cycle'].mean():.2f} €")
    print("Coût mensuel (somme des jours) :")
    display(daily_costs.to_frame("€/mois"))
else:
    print("Pas de colonne de coût detectée. Fournir PRICE_COL pour monétiser les économies.")


## 8) Synthèse interprétative (à compléter après exécution)
- Facteurs majeurs observés : …
- Plages horaires critiques : …
- Potentiel d'économie (kg / €) : …
- KPI candidats : consommation prédite vs réelle, surconsommation évitée, coût évité.
